<h1>TME 01 : Recommandation sociale</h1>

<h3>Importations des librairies utilisées</h3>

In [1]:
import math;
from math import exp
import random;
import numpy as np;

<h3>Fonctions de chargement des données</h3>

Les données de train ou de test seront de la forme : <br/>
{ <b>user1</b> : { movie1 : rating11, movie2 : rating12, .... }, <b>user2</b> : { .... } ...}

In [2]:
def loadMovieLens(path='ms/recodata', file='/u.data'):    
    prefs={};    
    for line in open(path+'/'+file):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})    
        prefs[user][movieid]=float(rating)
    return prefs

Les liens entre utilisateurs seront représentés sur un dictionnaire simple:<br/>
{ <b>user1</b> : { userVoisin1, userVoisin2, userVoisin3, ... }, <b>user2</b> : {...}  ...}

In [3]:
def loadLinks(path='ms/recodata', file='/u.links'):
    prefs={};
    for line in open(path+'/'+file):
        line  = line[:-2]        
        links = line.split('\t')
        prefs.setdefault(links[0],{})            
        prefs[links[0]] = links[1:len(links)]
    return prefs

<h3>Chargement des données</h3>
Nous avons testé notre modèle sur les données u1:

In [4]:
train1 = loadMovieLens("ms/recodata","u1.train");
test1 = loadMovieLens("ms/recodata","u1.test");
links = loadLinks();

La fonction suivante permet d'extraire les notes des utilisateurs sur des films sur la partie test. Elle sera utilisée à la fin pour pouvoir evaluer notre modèle

In [5]:
def getRates(test):
    rates = []
    for user in test:
        for item in test[user].keys():                                                
            rates.append(test[user][item])                
    return np.array(rates)

<h3>Modèle SoRec</h3>

Notre modèle sera donc representé par la classe SoRec définie ci-dessou, en plus des deux fonctions fit et predict, elle contient la fonction logistique g et sa dérivé g':

In [6]:
class SoRec():
    def __init__(self, n, I, eps, train, links):
        self.I       = I
        self.n       = n
        self.eps     = eps
        self.train   = train
        self.links   = links
        self.Z = {}
        self.W = {}
        self.F = {}
        self.getRandomVecs(train)
        
    # Methode qui entraine le modèle    
    def fit(self, train):
        e = []
        for iteration in range(self.I):            
            error1 = 0
            error2 = 0 
            for j in range(len(train)):
                
                Rui,u,i = self.getRandomUserItem(train)   
                while u not in links:
                    Rui,u,i = self.getRandomUserItem(train)
                    
                self.Z[u] = self.Z[u] - self.eps * (
                                                    ( Rui - self.g( self.Z[u].dot(self.W[i]) ) ) 
                                                    *self.dg(self.Z[u].dot(self.W[i])) 
                                                    *self.W[i]
                                                   )                             
                self.W[i] = self.W[i] - self.eps * (
                                                    (Rui - self.g( self.Z[u].dot(self.W[i])) )
                                                    *self.dg(self.Z[u].dot(self.W[i]))
                                                    *self.Z[u]
                                                    )                   
                error1 += (Rui - self.g(self.Z[u].dot(self.W[i])))**2
                                
                l = random.choice(list(links[u]))                              
                if l not in self.F:
                    self.F.setdefault(l,{})            
                    self.F[l] = np.random.rand(1,self.n)[0]
                r = np.random.random()                  
                self.Z[u] = self.Z[u] - self.eps * (
                                                    ( Rui - self.g( self.Z[u].dot(self.F[l]) ) ) 
                                                    *self.dg(self.Z[u].dot(self.F[l])) 
                                                    *self.F[l]
                                                   )                    
                self.F[l] = self.F[l] - self.eps * (
                                                    ( 1 - self.g( self.Z[u].dot(self.F[l])) )
                                                    *self.dg(self.Z[u].dot(self.F[l]))
                                                    *self.Z[u]
                                                   )
                            
                error2 += (1 - self.g(self.Z[u].dot(self.F[l])))**2            
            if iteration%10==0:    
                print str(iteration)," : ",(error1+error2)/len(train)
            e.append((error1+error2)/len(train))    
        """    
        import matplotlib.pyplot as plt
        plt.plot(e)
        plt.ylabel('some numbers')
        plt.show()    
        """
    # Méthode qui fait la prédiction
    def predict(self, test):
        prediction = []
        r=0
        for user in test:
            for item in test[user].keys():                                                
                if item in self.W:
                    r = self.Z[user].dot(self.W[item])
                    prediction.append(r)
                else:
                    prediction.append(r)
        return prediction
    
    
    
    ##### D'autres methodes de calcul:
    def g(self,x):
        return 1.0/( 1.0+exp(-x) )
    def dg(self,x):                
        return -exp(-x/( 1.0+exp(-x)))
    
    
    def getRandomVecs(self, train):        
        for t in train.keys():
            self.Z.setdefault(t,{})
            self.Z[t]= np.random.rand(1,self.n)[0]
            
        for t in train.values():
            for item in t.keys():
                self.W.setdefault(item,{})                                                  
                self.W[item]= np.random.rand(1,self.n)[0]                
                
    def getRandomUserItem(self, data):            
        u = random.choice(list(data.keys()))
        i = random.choice(list(data[u]))
        Rui = data[u][i]
        return Rui,u,i    

In [7]:
# Instanciation du modèle
model = SoRec(5, 100, 0.01, train1, links)
# Apprentissage du modèle
model.fit(train1)
# Prédiction du modèle
pred = model.predict(test1)
notes = getRates(test1)
# Calcul de l'erreur
mae = np.abs(pred - notes).mean()
print "mae : ",mae

0  :  9.1097651798
10  :  8.87205655026
20  :  8.85829650563
30  :  8.2904590095
40  :  8.13714748295
50  :  8.25362267883
60  :  8.41514086771
70  :  8.52027847202
80  :  8.22092572588
90  :  7.94626930537
mae :  0.98760394855


<h3>Modèle sur les autres données</h3>

In [8]:
strTrain = ['u1.train','u2.train','u3.train','u4.train','u5.train']
strTest  = ['u1.test','u2.test','u3.test','u4.test','u5.test']
links = loadLinks();
for i in range(5):
    print "Jeux de données numero ",(i+1)
    train = loadMovieLens("ms/recodata",strTrain[i]);
    test  = loadMovieLens("ms/recodata",strTest[i]);
    # Instanciation du modèle
    model = SoRec(5, 100, 0.01, train, links)
    # Apprentissage du modèle
    model.fit(train)
    # Prédiction du modèle
    pred = model.predict(test)
    notes = getRates(test)
    # Calcul de l'erreur
    mae = np.abs(pred - notes).mean()
    print "mae : ",mae
    print "------------------------------------------------"

Jeux de données numero  1
0  :  9.35965993118
10  :  8.88236730221
20  :  8.52677221394
30  :  8.56746691924
40  :  8.45345573997
50  :  8.36736674939
60  :  8.37895691792
70  :  7.65379168782
80  :  8.14604620276
90  :  8.12163960744
mae :  0.984884200087
------------------------------------------------
Jeux de données numero  2
0  :  9.29890202665
10  :  8.90510744048
20  :  8.65236963315
30  :  8.20280862437
40  :  8.12815342819
50  :  8.41507243293
60  :  8.24331036438
70  :  8.48124401025
80  :  7.99932352994
90  :  8.01160773587
mae :  0.989942958903
------------------------------------------------
Jeux de données numero  3
0  :  9.26436248784
10  :  8.76717715569
20  :  8.54172276386
30  :  8.41963803622
40  :  8.56055259193
50  :  8.34554973644
60  :  8.01702115285
70  :  8.18926334172
80  :  8.04992260777
90  :  8.02201065114
mae :  0.973792302177
------------------------------------------------
Jeux de données numero  4
0  :  9.53172406269
10  :  8.44860624376
20  :  8.635579